In [1]:
import logging
import time
import warnings

import pandas as pd
import torch
import tqdm
from optuna.exceptions import ExperimentalWarning
from pyspark.sql import functions as sf

from replay.data_preparator import DataPreparator, Indexer
from replay.experiment import Experiment
from replay.metrics import HitRate, NDCG, MAP, MRR, Coverage, Surprisal
from replay.models import ALSWrap, KNN, LightFMWrap, SLIM, UCB, CQL, Wilson
from replay.session_handler import State
from replay.splitters import DateSplitter
from replay.utils import get_log_info

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=ExperimentalWarning)
use_gpu = torch.cuda.is_available()

/Users/kuderov/mambaforge/envs/offrl/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


## Set Spark log level

In [2]:
spark = State().session
spark.sparkContext.setLogLevel('ERROR')

22/08/01 11:51:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/01 11:51:05 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/08/01 11:51:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Read ML1M dataset

In [3]:
prefix = "./data/"
df_log = pd.read_csv(f'{prefix}/ml1m_ratings.dat', sep='\t', names=['user_id', 'item_id', 'relevance', 'timestamp'])
# df_items = pd.read_csv(f'{prefix}/ml1m_items.dat', sep='\t', names=['item_id', 'name', 'genre'])
# df_users = pd.read_csv(f'{prefix}/ml1m_users.dat', sep='\t', names=['user_id', 'gender', 'age', 'occupation', 'zip_code'])

In [4]:
col_mapping = {key: key for key in ['user_id', 'item_id', 'relevance', 'timestamp']}

data_preparator = DataPreparator()
log = data_preparator.transform(columns_mapping=col_mapping, data=df_log)

01-Aug-22 11:51:07, replay, INFO: Columns with ids of users or items are present in mapping. The dataframe will be treated as an interactions log.
/Users/kuderov/mambaforge/envs/offrl/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:34: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):


In [5]:
log.show(2)

+-------+-------+---------+-------------------+
|user_id|item_id|relevance|          timestamp|
+-------+-------+---------+-------------------+
|      1|   1193|      5.0|2001-01-01 01:12:40|
|      1|    661|      3.0|2001-01-01 01:35:09|
+-------+-------+---------+-------------------+
only showing top 2 rows



In [6]:
log.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- item_id: long (nullable = true)
 |-- relevance: double (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [7]:
get_log_info(log, user_col='user_id', item_col='item_id')

'total lines: 1000209, total users: 6040, total items: 3706'

Experiment constants/hyperparams

In [8]:
K = 10
K_list_metrics = [1, 5, 10]
SEED = 12345

## Prepare data for training/testing

Apply PySpark indexing, then split randomly into train/test datasets

In [9]:
indexer = Indexer()
indexer.fit(users=log.select('user_id'), items=log.select('item_id'))

In [10]:
# will consider ratings >= 3 as positive feedback. A positive feedback is treated with relevance = 1
only_positives_log = log.filter(sf.col('relevance') >= 3).withColumn('relevance', sf.lit(1.))
# negative feedback will be used for Wilson and UCB models
only_negatives_log = log.filter(sf.col('relevance') < 3).withColumn('relevance', sf.lit(0.))

only_positives_log.count(), only_negatives_log.count()

(836478, 163731)

In [11]:
pos_log = indexer.transform(df=only_positives_log)
pos_log.show(2)

/Users/kuderov/mambaforge/envs/offrl/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:34: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):


+--------+--------+---------+-------------------+
|user_idx|item_idx|relevance|          timestamp|
+--------+--------+---------+-------------------+
|    4131|      43|      1.0|2001-01-01 01:12:40|
|    4131|     585|      1.0|2001-01-01 01:35:09|
+--------+--------+---------+-------------------+
only showing top 2 rows



In [12]:
# train/test split 
date_splitter = DateSplitter(
    test_start=0.2,
    drop_cold_items=True,
    drop_cold_users=True,

)
train, test = date_splitter.split(pos_log)
train.cache(), test.cache()
print('train info:\n', get_log_info(train))
print('test info:\n', get_log_info(test))

train info:
 total lines: 669181, total users: 5397, total items: 3569


test info:
 total lines: 86542, total users: 1139, total items: 3279


In [13]:
test_start = test.agg(sf.min('timestamp')).collect()[0][0]

# train with both positive and negative feedback
pos_neg_train=(
    train
    .withColumn('relevance', sf.lit(1.))
    .union(
        indexer.transform(
            only_negatives_log.filter(sf.col('timestamp') < test_start)
        )
    )
)
pos_neg_train.cache()
pos_neg_train.count()

/Users/kuderov/mambaforge/envs/offrl/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:34: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/Users/kuderov/mambaforge/envs/offrl/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:34: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):


798993

## Run training

In [14]:
def fit_predict_add_res(name, model, experiment, train, top_k, test_users):
    """
    Run fit_predict for the `model`, measure time on fit_predict and evaluate metrics
    """
    start_time=time.time()
    
    model.fit(log=train)
    fit_time = time.time() - start_time

    pred=model.predict(log=train, k=top_k, users=test_users)
    pred.cache()
    pred.count()
    predict_time = time.time() - start_time - fit_time

    experiment.add_result(name, pred)
    metric_time = time.time() - start_time - fit_time - predict_time
    
    experiment.results.loc[name, 'fit_time'] = fit_time
    experiment.results.loc[name, 'predict_time'] = predict_time
    experiment.results.loc[name, 'metric_time'] = metric_time
    experiment.results.loc[name, 'full_time'] = (fit_time + predict_time + metric_time)
    pred.unpersist()

In [15]:
experiment = Experiment(test, {MAP(): K, NDCG(): K, HitRate(): K_list_metrics, Coverage(train): K, Surprisal(train): K, MRR(): K})

algorithms = {
    'CQL': CQL(use_gpu=use_gpu, k=K, n_epochs=3),
    'ALS': ALSWrap(seed=SEED),
    'KNN': KNN(num_neighbours=K),
    'LightFM': LightFMWrap(random_state=SEED), 
    'SLIM': SLIM(seed=SEED),
    'UCB': UCB(exploration_coef=0.5)
}

01-Aug-22 11:52:02, replay, INFO: The model is neural network with non-distributed training


In [16]:
%%time
logger = logging.getLogger("replay")
test_users = test.select('user_idx').distinct()

for name in tqdm.auto.tqdm(algorithms.keys(), desc='Model'):
    model = algorithms[name]
    
    logger.info(msg='{} started'.format(name))
    
    train_ = train
    if isinstance(model, (Wilson, UCB)):
        train_ = pos_neg_train
    fit_predict_add_res(name, model, experiment, train=train_, top_k=K, test_users=test_users)
    print(experiment.results[['NDCG@{}'.format(K), 'MRR@{}'.format(K), 'Coverage@{}'.format(K), 'fit_time']].sort_values('NDCG@{}'.format(K), ascending=False))

Model:   0%|          | 0/6 [00:00<?, ?it/s]

01-Aug-22 11:52:02, replay, INFO: CQL started
/Users/kuderov/mambaforge/envs/offrl/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:34: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/Users/kuderov/mambaforge/envs/offrl/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:34: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):


2022-08-01 11:52.10 [debug    ] RoundIterator is selected.
2022-08-01 11:52.10 [info     ] Directory is created at d3rlpy_logs/CQL_20220801115210
2022-08-01 11:52.10 [debug    ] Building models...
2022-08-01 11:52.10 [debug    ] Models have been built.
2022-08-01 11:52.10 [info     ] Parameters are saved to d3rlpy_logs/CQL_20220801115210/params.json params={'action_scaler': None, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'actor_learning_rate': 0.0001, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_learning_rate': 0.0001, 'alpha_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_threshold': 10.0, 'batch_size': 256, 'conservative_weight': 5.0, 'critic_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rat

Epoch 1/3:   0%|          | 0/2613 [00:00<?, ?it/s]

2022-08-01 11:54.19 [info     ] CQL_20220801115210: epoch=1 step=2613 epoch=1 metrics={'time_sample_batch': 0.0005153620603483297, 'time_algorithm_update': 0.04833759907511884, 'temp_loss': -8.16541134468695, 'temp': 1.0999085514015317, 'alpha_loss': -3.906728253722561, 'alpha': 1.1031546600924294, 'critic_loss': 1043.9835001169563, 'actor_loss': -85.92551019216377, 'time_step': 0.04899401378230121} step=2613
2022-08-01 11:54.19 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220801115210/model_2613.pt


...

/Users/kuderov/mambaforge/envs/offrl/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:34: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/Users/kuderov/mambaforge/envs/offrl/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:34: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/Users/kuderov/mambaforge/envs/offrl/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(
/Users/kuderov/mambaforge/envs/offrl/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(
/Users/kuderov/mambaforge/envs/offrl/lib/python3.9/site-pa

          NDCG@10    MRR@10  Coverage@10    fit_time
ALS      0.255471  0.413317     0.147380   20.821820
KNN      0.246561  0.404074     0.102550   40.617612
LightFM  0.238080  0.395384     0.298123   39.239888
CQL      0.033492  0.078719     0.207341  393.981391


/Users/kuderov/mambaforge/envs/offrl/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:34: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/Users/kuderov/mambaforge/envs/offrl/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:34: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
01-Aug-22 12:04:20, replay, INFO: UCB started                                   


          NDCG@10    MRR@10  Coverage@10    fit_time
ALS      0.255471  0.413317     0.147380   20.821820
KNN      0.246561  0.404074     0.102550   40.617612
SLIM     0.239382  0.414097     0.149902   26.352801
LightFM  0.238080  0.395384     0.298123   39.239888
CQL      0.033492  0.078719     0.207341  393.981391


          NDCG@10    MRR@10  Coverage@10    fit_time
ALS      0.255471  0.413317     0.147380   20.821820
KNN      0.246561  0.404074     0.102550   40.617612
SLIM     0.239382  0.414097     0.149902   26.352801
LightFM  0.238080  0.395384     0.298123   39.239888
CQL      0.033492  0.078719     0.207341  393.981391
UCB      0.000193  0.000878     0.019894   10.527998
CPU times: user 6min 52s, sys: 3min 2s, total: 9min 54s
Wall time: 13min 5s


In [17]:
experiment.results.sort_values('NDCG@10', ascending=False)

,Coverage@10,HitRate@1,HitRate@5,HitRate@10,MAP@10,MRR@10,NDCG@10,Surprisal@10,fit_time,predict_time,metric_time,full_time
ALS,0.147380,0.303775,0.569798,0.673398,0.163541,0.413317,0.255471,0.170315,20.821820,31.702029,19.725200,72.249049
KNN,0.102550,0.290606,0.560140,0.650571,0.156526,0.404074,0.246561,0.164692,40.617612,16.521744,19.747590,76.886946
SLIM,0.149902,0.298507,0.569798,0.668130,0.144292,0.414097,0.239382,0.176250,26.352801,19.759773,13.910726,60.023300
LightFM,0.298123,0.268657,0.553995,0.677788,0.144336,0.395384,0.238080,0.208283,39.239888,23.265942,13.916489,76.422319
CQL,0.207341,0.031607,0.146620,0.208955,0.013220,0.078719,0.033492,0.455415,393.981391,31.513767,26.960444,452.455602
UCB,0.019894,0.000878,0.000878,0.000878,0.000088,0.000878,0.000193,1.000000,10.527998,18.765398,17.631306,46.924702
